In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from tkinter import Tk, Label, Entry, Button, StringVar

# Đọc dữ liệu và chuẩn bị mô hình
df = pd.read_csv("heart_2020_cleaned.csv")
df["HeartDisease"] = df["HeartDisease"].map({"Yes": 1, "No": 0})
categorical_cols = df.select_dtypes(exclude=["number"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ],
    remainder="passthrough",
)

X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter=1000)),
    ]
)
model.fit(X_train, y_train)

# Tạo giao diện người dùng
root = Tk()
root.title("Heart Disease Prediction")

# Lưu trữ các nhãn và ô nhập liệu
entries = {}
for idx, col in enumerate(X.columns):
    Label(root, text=col).grid(row=idx, column=0)
    entries[col] = Entry(root)
    entries[col].grid(row=idx, column=1)

# Kết quả chẩn đoán
result_var = StringVar()
Label(root, text="Prediction Result:").grid(row=len(X.columns), column=0)
Label(root, textvariable=result_var).grid(row=len(X.columns), column=1)


def predict():
    # Lấy giá trị từ các ô nhập liệu
    input_data = {col: [entries[col].get()] for col in X.columns}
    input_df = pd.DataFrame(input_data)
    prediction = model.predict(input_df)

    if prediction[0] == 1:
        result_var.set("Heart Disease Detected")
    else:
        result_var.set("No Heart Disease Detected")


# Nút dự đoán
Button(root, text="Predict", command=predict).grid(row=len(X.columns) + 1, column=1)

root.mainloop()